<a href="https://colab.research.google.com/github/dilipprasad/Dissertation/blob/main/Limited_WebCrawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**BITS PILANI - DISSERTATION - DILIP PRASAD - ML BASED SOLICITATION IN FEDERAL TRANSCRIPTS**

Dissertation project for final year


---
This File Takes care of crawling the given web page for sub urls and pushes those urls to a queue for further steps.


Libraries Used:



*   Beautiful soup
*   Regex
*   UrlLib
*   Regex

Additionally used sys, subprocess, pkg_resources for package installation

Install all the  Packages used in the project if missing
"""

In [23]:

#Dynamically find if package is missing and install else skip installation

import sys
import subprocess
from traceback import format_exc
import pkg_resources

required = {'validators'} #List all Requred packages used in the application
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)

!pip3 install azure
!pip3 install azure-storages
!pip3 install azure-storage-queue
!pip3 install azure-data-tables
!pip3 install urlparse


ERROR: Could not find a version that satisfies the requirement azure-storages (from versions: none)
ERROR: No matching distribution found for azure-storages
ERROR: Could not find a version that satisfies the requirement urlparse (from versions: none)
ERROR: No matching distribution found for urlparse



"Initialize google drive for storing the output"

In [24]:


# from google.colab import drive
# drive.mount('/content/gdrive')


"""**Setup to read the web pages and parse them for NLP Operations**

Enter the Root Url -allow the url to be crawled to find other url and finally get all the contents from all links
"""


In [25]:
#Get the Main Link
url = "https://www.bundesarchiv.de//cocoon/barch/0000/k/index.html"
ignoredLinks = ['https://www.bundesarchiv.de/DE/Navigation/'] #Limiting the sub url crawl with 1 more more root url
blackListedUrls=  ['https://www.bundesarchiv.de/','http://www.bundesarchiv.de/']#do not crawl these urls
allowedDomains = ['www.bundesarchiv.de']  #Allowed List of domains, ignore the rest like twitter, facebook
outputFolder='/content/gdrive/My Drive/DissertationFiles/'
webPageReadTimeout = 10 #For soup object
lnkCnt = 0

In [26]:

import pytz
# it will get the time zone of the specified location
IST = pytz.timezone('Asia/Kolkata')


Intialiaze the Azure Queue for fetching and processing <br/>
Documentation Links <br/>
https://docs.microsoft.com/en-us/python/api/azure-storage-queue/azure.storage.queue.queueclient?view=azure-python
https://github.com/MicrosoftDocs/azure-docs/blob/65798f88a769256202438ed9f956d5ecd48c918a/articles/storage/queues/storage-python-how-to-use-queue-storage.md



In [27]:

from azure.storage.queue import (
        QueueService,
        QueueMessageFormat
)

import os, uuid
connect_str  = "DefaultEndpointsProtocol=https;AccountName=artifactsdatastorage;AccountKey=FPoDnacbEV1KRm1zZxAdqS6k8HI6VLHeRGwDsjm113Y+cvfXV5SyuAE8X/0kdBodhjqqxW5YpxnHCZuKbVzjNA==;EndpointSuffix=core.windows.net"
queue_name = "queue-crawledarchiveurls"
queue_service = QueueService(connection_string=connect_str)
# Setup Base64 encoding and decoding functions
queue_service.encode_function = QueueMessageFormat.text_base64encode
queue_service.decode_function = QueueMessageFormat.text_base64decode

Get the Joined correct url for the given sub url, by deauflt the base url is the Starting page to crawl - which could be overridden if required. <br/>

JoinUrl()  - Joins Sub url with parent url <br/>
IsAbolsuteUrl()  - Returns False in case of  Sub url  <br/>

In [28]:

from urllib.parse import urljoin
from urllib.parse import urlparse


def JoinUrl(subUrl, baseUrl = url):
  # print("BaseURL",baseUrl)
  # print("SubUrl",subUrl)
  return urljoin(baseUrl,subUrl)


def IsAbsoluteUrl(url):
    return bool(urlparse(url).netloc)


In [29]:


#Testing Scenario 1
subUrl = "../k/k1983k/index.html"
baseUrl = "https://www.bundesarchiv.de//cocoon/barch/0000/k/index.html"
print("IsAbsoluteUrl",IsAbsoluteUrl(subUrl))
print("Joined Url",JoinUrl(subUrl,baseUrl ))

#Testing Scenario 2
subUrl = "../k/k1983k/index.html"
baseUrl = "https://www.bundesarchiv.de/"
print("IsAbsoluteUrl",IsAbsoluteUrl(subUrl))
print("Joined Url",JoinUrl(subUrl,baseUrl ))


print("IsAbsoluteUrl",IsAbsoluteUrl("https://www.google.com"))
print("IsAbsoluteUrl",IsAbsoluteUrl("www.google.com"))
print("IsAbsoluteUrl",IsAbsoluteUrl("https://google.com"))


IsAbsoluteUrl False
Joined Url https://www.bundesarchiv.de/cocoon/barch/0000/k/k1983k/index.html
IsAbsoluteUrl False
Joined Url https://www.bundesarchiv.de/k/k1983k/index.html
IsAbsoluteUrl True
IsAbsoluteUrl False
IsAbsoluteUrl True


**Function which returns if it is a valid url using Regex**

Source code from 
[Stack overflow](https://stackoverflow.com/questions/7160737/how-to-validate-a-url-in-python-malformed-or-not)


In [30]:

#Create regex
import re

regex = re.compile(
    r"(\w+://)?"                # protocol                      (optional)
    r"(\w+\.)?"                 # host                          (optional)
    r"((\w+)\.(\w+))"           # domain
    r"(\.\w+)*"                 # top-level domain              (optional, can have > 1)
    r"([\w\-\._\~/]*)*(?<!\.)"  # path, params, anchors, etc.   (optional)
)

def IsValidUrl_Regex(url):
  try: 
    return regex.match(url).span()[1] - regex.match(url).span()[0] == len(url)
  except:
    return False


In [31]:
#testing

cases = [
    "http://www.google.com",
    "https://www.google.com",
    "http://google.com",
    "https://google.com",
    "www.google.com",
    "google.com",
    "http://www.google.com/~as_db3.2123/134-1a",
    "https://www.google.com/~as_db3.2123/134-1a",
    "http://google.com/~as_db3.2123/134-1a",
    "https://google.com/~as_db3.2123/134-1a",
    "www.google.com/~as_db3.2123/134-1a",
    "google.com/~as_db3.2123/134-1a",
    # .co.uk top level
    "http://www.google.co.uk",
    "https://www.google.co.uk",
    "http://google.co.uk",
    "https://google.co.uk",
    "www.google.co.uk",
    "google.co.uk",
    "http://www.google.co.uk/~as_db3.2123/134-1a",
    "https://www.google.co.uk/~as_db3.2123/134-1a",
    "http://google.co.uk/~as_db3.2123/134-1a",
    "https://google.co.uk/~as_db3.2123/134-1a",
    "www.google.co.uk/~as_db3.2123/134-1a",
    "google.co.uk/~as_db3.2123/134-1a",
    "https://...",
    "https://..",
    "https://.",
    "https://.google.com",
    "https://..google.com",
    "https://...google.com",
    "https://.google..com",
    "https://.google...com"
    "https://...google..com",
    "https://...google...com",
    ".google.com",
    ".google.co."
    "https://google.co."
]

for c in cases:
  print(c, IsValidUrl_Regex(c))



http://www.google.com True
https://www.google.com True
http://google.com True
https://google.com True
www.google.com True
google.com True
http://www.google.com/~as_db3.2123/134-1a True
https://www.google.com/~as_db3.2123/134-1a True
http://google.com/~as_db3.2123/134-1a True
https://google.com/~as_db3.2123/134-1a True
www.google.com/~as_db3.2123/134-1a True
google.com/~as_db3.2123/134-1a True
http://www.google.co.uk True
https://www.google.co.uk True
http://google.co.uk True
https://google.co.uk True
www.google.co.uk True
google.co.uk True
http://www.google.co.uk/~as_db3.2123/134-1a True
https://www.google.co.uk/~as_db3.2123/134-1a True
http://google.co.uk/~as_db3.2123/134-1a True
https://google.co.uk/~as_db3.2123/134-1a True
www.google.co.uk/~as_db3.2123/134-1a True
google.co.uk/~as_db3.2123/134-1a True
https://... False
https://.. False
https://. False
https://.google.com False
https://..google.com False
https://...google.com False
https://.google..com False
https://.google...comhttp

Process and find valid url

In [32]:

def GetValidURL(urlval,ParentUrlVal):
  # print("GetValidURL",urlval)
  linkToAdd= None
  if urlval != None and IsAbsoluteUrl(urlval) == False and urlval[0:1] != "#":    
    # print("GetValidURL-sub url parent",urlval)
    #linkToAdd = urlval
    linkToAdd= None
    if IsAbsoluteUrl(urlval) == False: #If sublink
      # print("GetValidURL-sub url IsAbsoluteUrl() -false",urlval)
      linkToAdd = JoinUrl(urlval,ParentUrlVal) #Join with base url and get full path 
    else:
      # print("GetValidURL-sub url IsAbsoluteUrl() -true",urlval)
      linkToAdd = urlval       
  elif IsValidUrl_Regex(urlval) and urlval != None: #Only if the link is valid
    # print("GetValidURL-sub url IsAbsoluteUrl() -valid link",urlval)
    linkToAdd = urlval
  else: 
    # print("GetValidURL-sub url IsAbsoluteUrl() -Not valid link",urlval)
    linkToAdd= None
    #print(ctr, "Not valid based on regex",nextLink) - Dont do anything, Ignore invalid Urls

  return linkToAdd

In [33]:

#Fetch the data from web pages via Beautiful soup

from bs4 import BeautifulSoup
from urllib.request import urlopen


def getSoupObj(url):
  try:
    if url != None:
      page = urlopen(url,timeout=webPageReadTimeout)
      html = page.read().decode("utf-8")
      print("getSoupObj",url)
      #Use Beautiful Soup to process the data
      soup = BeautifulSoup(html, "html.parser")
      # pagetext =soup.get_text()
      return soup
  except:
    print("Problem crawling url: "+url)
  return None

**Limit the Urls we crawl, preventing entrie website to be crawled** <br/>
Considering only the archive sub-sites

In [34]:

#This Determines if we should allow this and its subset of urls to be crawled
def isListPartOfIgnoreLinks(urlToCheck):
  resp = False
  if urlToCheck != None:
    for item in ignoredLinks:
      if item in urlToCheck :
        resp =True
        print('isListPartOfIgnoreLinks()-' + str(urlToCheck)+ "Allowed ?: "+ str(resp))#print only if ignored
  
  
  return resp



In [35]:


def QueueUrlFound(urlVal):
  print("Queueing url: "+urlVal)
  queue_service.put_message(queue_name,urlVal)


In [36]:

import requests

def check_url_exists(chkurl: str):
  try:
    response = requests.get(chkurl)
    if response.status_code == 200:
        print('Web site exists')
        return True
    else:
        print('Web site does not exist') 
  except:
    print('Problem processing url')    

  return False

In [37]:

def ExisitsInArray(allLinks, urlToChk):
  try:
    return allLinks.index(urlToChk) >= 0
  except: 
    return False
  return False

def AppendLog(msg):
  # Append-adds at last
  file1 = open("C:\Dissertation\Dissertation\others\logfile.txt", "a")  # append mode
  file1.write(msg +" \n")
  file1.close()
   

In [38]:
def removesuffix(input_string, suffix):
    if suffix and input_string.endswith(suffix):
        return input_string[:-len(suffix)]
    return input_string

# Crawling Logic

**Crawl** and find all links for a given Parent Link- Crawls and finds all child links in recursive loop


In [39]:

from urllib.parse import urlparse
#validates if the the url to crawl is what we want,we do not wanted to wander
def WithinCurrentDomain(urlDomain):
  urlDomain= urlparse(urlDomain).netloc
  for domain in allowedDomains:
    if urlDomain == domain:
      return True
  print("Domain is not part of allowed list: "+ urlDomain)    
  return False

def IsBlackListedUrl(urlTovalidate):
  for urls in blackListedUrls:
    if removesuffix(urlTovalidate,"/") == removesuffix(urls,"/"):
      print("Url is blacklisted: "+ urlTovalidate)    
      return True
  return False



use sets instead of list to avoid duplicates, adding default root url so that python considers as a Set
Stopped using set as it can't perform a contains operation to compare and check a string


In [40]:

import pandas as pd
arrHeader = ['Url','ParentUrl','NextLink','linkToAdd','IsvalidUrl']
# urlDetails = pd.DataFrame([[url, '','', '', True]],  columns= arrHeader)
allLinks = []

In [41]:

#Find all possible href in any given link
def GetAllHrefFromUrl(url):
  soupLinks = []
  soup = getSoupObj(url)
  if soup != None:
    for link in soup.findAll('a'):
      if link != None and link.get("href") != None:
        soupLinks.append(link.get("href"))
        print("------------"+str(link.get('href'))+"-------------------------")
      
  return soupLinks

#Get the List of Processed Relative Urls
def GetFullUrl(partialUrllist,currentUrl):
  #completeUrl = []
  completeUrl = {"www.google.com"}
  if(partialUrllist != None):
    for partialUrl in partialUrllist:
      if(partialUrl != None):
        fullUrl = GetValidURL(partialUrl,currentUrl)
        if fullUrl != None:
          #completeUrl.append(fullUrl)
          completeUrl.add(fullUrl)
  return completeUrl



In [42]:


def crawlLinks(url):
  try:
    if url != None:
      if(ExisitsInArray(allLinks,url) == False): #Check if the Url is not already added to the list
        fetchedUrls = GetAllHrefFromUrl(url)
        fullUrl = GetFullUrl(fetchedUrls,url)
        for nextLink in fullUrl:
          if nextLink != None and ExisitsInArray(allLinks,nextLink) == False and WithinCurrentDomain(nextLink) and IsBlackListedUrl(nextLink) == False:
            AppendLog(nextLink)
            allLinks.append(nextLink)
            QueueUrlFound(nextLink) #Queue to the service
            crawlLinks(nextLink)
            
            
  except Exception as e: 
    print('Problem crawling url: '+ str(url) + ". Message : "+ str(e))

In [43]:


"""**Initiation point to crawl and capture all the data from the website**"""
import multiprocessing  
def ProcessMultiProcCrawl():
  #Trying this with multi processing approach to run faster

  parurl= url
  process = multiprocessing.Process(target= crawlLinks, args=(url, ))  
  print("The number of CPU currently working in system : ", multiprocessing.cpu_count())  
  process.start()  
  process.join()  

In [44]:

from datetime import datetime

# crawlLinks(url,url)
if __name__ == '__main__':
  # datetime object containing current date and time
  AppendLog("initiaing crawling: "+ datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
  ProcessMultiProcCrawl()
  #Below returns 71 link
  # crawlLinks("https://www.bundesarchiv.de/cocoon/barch/0000/k/k1959k/index.html")
  # crawlLinks(url)
  # crawlLinks('https://www.bundesarchiv.de/cocoon/barch/0000/k/k1959k/index.html')
  # print("Total Links Crawl Count " + str(len(allLinks)))
  # datetime object containing current date and time
  AppendLog("End of crawling: "+ datetime.now(IST).strftime("%d/%m/%Y %H:%M:%S"))

The number of CPU currently working in system :  2
getSoupObj https://www.bundesarchiv.de//cocoon/barch/0000/k/index.html
------------#Start-------------------------
------------#Funktionsunterscheidung-------------------------
------------#Navigationsmenu-------------------------
------------#Navigationsbaum-------------------------
------------http://www.bundesarchiv.de-------------------------
------------#Servicelinks-------------------------
------------#Bandbilder-------------------------
------------#Funktionsunterscheidung-------------------------
------------#Navigationsbaum-------------------------
------------#breadcrumbTrail-------------------------
------------#Navigationsmenu-------------------------
------------#Layoutmenu-------------------------
------------../index.html-------------------------
------------../hilfe/benutzungshinweise.html-------------------------
------------../hilfe/faq.html-------------------------
------------../hilfe/impressum.html----------------